# sophia_lda_model_v2 (use more restaurants)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

In [3]:
key_phrases = pd.read_csv('./csv/topic_allocation_with_labels.csv')
key_phrases


,phrase,score,topic,topic_label
0,customer service,192.945282,0,Customer Service Experience
1,of,161.413885,0,Customer Service Experience
2,room,153.289571,0,Customer Service Experience
3,not,150.768080,0,Customer Service Experience
4,customer,106.777045,0,Customer Service Experience
...,...,...,...,...
295,line,89.368565,29,Tourist Avoidance
296,long line,82.316417,29,Tourist Avoidance
297,much,79.759205,29,Tourist Avoidance
298,food mediocre,78.052394,29,Tourist Avoidance


In [4]:

n_topics = key_phrases['topic'].nunique()
n_topics

30

In [148]:
df = pd.read_csv('./csv/df_review_top10.csv')
df.stars.mean(), df.stars.min(), df.stars.max(), df.shape

(4.0409114937320645, 1.0, 5.0, (52968, 10))

In [149]:
df

,review_id,business_id,stars,name,text,restaurant_avg_star,review_count,avg_stars10m_radius,date,clean_text
0,yyrMqY7sNp5gT7KJ1AaYWA,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,We have been here twice for brunch and have en...,4.0,4554,4.05814,2016-03-07,twice brunch enjoyed immensely everything trie...
1,wEfzqOfbwn4Ohe2ZDOLAzw,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,First meal in New Orleans. I had the $15 lunch...,4.0,4554,4.05814,2012-11-06,first meal new orleans lunch special seafood ...
2,XT_3UpEhO5eJIFxevnv_Yw,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,Service is excellent. Atmosphere is great. The...,4.0,4554,4.05814,2015-02-02,service excellent atmosphere great raw bar fan...
3,YpHIixkA2K96wFLbI6FHkQ,GBTPC53ZrG1ZBY3DT8Mbcw,5.0,Luke,This was our second choice after finding out R...,4.0,4554,4.05814,2009-03-01,second choice finding rio mar closed mardi gra...
4,gnEmBGyBs7DOtIXGmJQdKQ,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,Great oyster happy hours that go on from 5-7pm...,4.0,4554,4.05814,2015-08-19,great oyster happy hours go pm boyfriend share...
...,...,...,...,...,...,...,...,...,...,...
52963,CqJxh0TxfV2kb7ny5jeR0g,oBNrLz4EDhiscSlbOl8uAw,4.0,Ruby Slipper - New Orleans,Heard a lot about this place. I was told by ma...,4.5,5193,4.05814,2017-02-20,heard lot place told many including yelp nola ...
52964,Qq3hOwfgkJG8CL6B39lHRA,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,I don't even know where to start. This was ou...,4.5,5193,4.05814,2021-12-13,even know start top recommendation local frie...
52965,EqXPcN4t6tqKE47tBdUX_Q,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,Long wait and they got my phone number wrong s...,4.5,5193,4.05814,2021-12-04,long wait got phone number wrong even longer e...
52966,HehXCiDy2-vwX94in-_Zcw,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,This place was amazing! From the host taking ...,4.5,5193,4.05814,2022-01-16,place amazing host taking names servers forge...


In [150]:
# Add 20 new columns with names "topics1" to "topics20"
for i in range(1, n_topics + 1):
    col_name = topic_labels[i - 1]
    df[col_name] = None
df.columns

Index(['review_id', 'business_id', 'stars', 'name', 'text',
       'restaurant_avg_star', 'review_count', 'avg_stars10m_radius', 'date',
       'clean_text', 'Customer Service Experience', 'Bar Atmosphere',
       'Mediocre Food', 'Lacking Value', 'New Orleans Culture', 'Long Waits',
       'Food Quality', 'Southern Dishes', 'Shrimp Po' Boy',
       'Returning Doubtful', 'Taste Preferences', 'Southern Comfort Food',
       'Happy Hour Selection', 'Cheese and Steak', 'Philadelphia Cheesesteak',
       'Reliable Staff', 'Quality Dining', 'Outstanding Service',
       'Satisfactory Experience', 'Dissatisfied Feedback', 'Return Intent',
       'Southern Classics', 'Outdoor Seating', 'Service Issues',
       'Extraordinary Recommendation', 'Oyster Specialties',
       'Breakfast Choices', 'Favorite Spot', 'Top Picks', 'Tourist Avoidance'],
      dtype='object')

In [151]:
df

,review_id,business_id,stars,name,text,restaurant_avg_star,review_count,avg_stars10m_radius,date,clean_text,...,Return Intent,Southern Classics,Outdoor Seating,Service Issues,Extraordinary Recommendation,Oyster Specialties,Breakfast Choices,Favorite Spot,Top Picks,Tourist Avoidance
0,yyrMqY7sNp5gT7KJ1AaYWA,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,We have been here twice for brunch and have en...,4.0,4554,4.05814,2016-03-07,twice brunch enjoyed immensely everything trie...,...,None,None,None,None,None,None,None,None,None,None
1,wEfzqOfbwn4Ohe2ZDOLAzw,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,First meal in New Orleans. I had the $15 lunch...,4.0,4554,4.05814,2012-11-06,first meal new orleans lunch special seafood ...,...,None,None,None,None,None,None,None,None,None,None
2,XT_3UpEhO5eJIFxevnv_Yw,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,Service is excellent. Atmosphere is great. The...,4.0,4554,4.05814,2015-02-02,service excellent atmosphere great raw bar fan...,...,None,None,None,None,None,None,None,None,None,None
3,YpHIixkA2K96wFLbI6FHkQ,GBTPC53ZrG1ZBY3DT8Mbcw,5.0,Luke,This was our second choice after finding out R...,4.0,4554,4.05814,2009-03-01,second choice finding rio mar closed mardi gra...,...,None,None,None,None,None,None,None,None,None,None
4,gnEmBGyBs7DOtIXGmJQdKQ,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,Great oyster happy hours that go on from 5-7pm...,4.0,4554,4.05814,2015-08-19,great oyster happy hours go pm boyfriend share...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52963,CqJxh0TxfV2kb7ny5jeR0g,oBNrLz4EDhiscSlbOl8uAw,4.0,Ruby Slipper - New Orleans,Heard a lot about this place. I was told by ma...,4.5,5193,4.05814,2017-02-20,heard lot place told many including yelp nola ...,...,None,None,None,None,None,None,None,None,None,None
52964,Qq3hOwfgkJG8CL6B39lHRA,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,I don't even know where to start. This was ou...,4.5,5193,4.05814,2021-12-13,even know start top recommendation local frie...,...,None,None,None,None,None,None,None,None,None,None
52965,EqXPcN4t6tqKE47tBdUX_Q,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,Long wait and they got my phone number wrong s...,4.5,5193,4.05814,2021-12-04,long wait got phone number wrong even longer e...,...,None,None,None,None,None,None,None,None,None,None
52966,HehXCiDy2-vwX94in-_Zcw,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,This place was amazing! From the host taking ...,4.5,5193,4.05814,2022-01-16,place amazing host taking names servers forge...,...,None,None,None,None,None,None,None,None,None,None


In [152]:
# Initialize the new columns with zeros
for i in range(1, n_topics + 1):
    col_name = topic_labels[i-1]
    df[col_name] = 0.0  # Initialize with 0.0 (or any other default value)

# Iterate through each row of the dataframe
for index, row in df.iterrows():
    clean_text = row['clean_text']

    # Iterate through each phrase
    for phrase, score, topic in zip(key_phrases['phrase'], key_phrases['score'], key_phrases['topic']):
        # Check if the phrase appears in the clean_text
        if phrase in clean_text:
            # If it does, sum the score to the corresponding topic
            df.at[index, topic_labels[topic]] +=  score

In [147]:
df

,review_id,business_id,stars,name,text,restaurant_avg_star,review_count,avg_stars10m_radius,date,clean_text,...,Return Intent,Southern Classics,Outdoor Seating,Service Issues,Extraordinary Recommendation,Oyster Specialties,Breakfast Choices,Favorite Spot,Top Picks,Tourist Avoidance
0,yyrMqY7sNp5gT7KJ1AaYWA,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,We have been here twice for brunch and have en...,4.0,4554,4.05814,2016-03-07,twice brunch enjoyed immensely everything trie...,...,0.000000,0.000000,0.000000,322.624927,0.000000,0.000000,86.084291,0.000000,68.974594,0.000000
1,wEfzqOfbwn4Ohe2ZDOLAzw,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,First meal in New Orleans. I had the $15 lunch...,4.0,4554,4.05814,2012-11-06,first meal new orleans lunch special seafood ...,...,0.000000,0.000000,0.000000,317.114038,89.446985,156.089689,0.000000,0.000000,91.116914,0.000000
2,XT_3UpEhO5eJIFxevnv_Yw,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,Service is excellent. Atmosphere is great. The...,4.0,4554,4.05814,2015-02-02,service excellent atmosphere great raw bar fan...,...,0.000000,156.992318,77.808380,645.076282,201.488593,0.000000,0.000000,0.000000,258.248579,107.976308
3,YpHIixkA2K96wFLbI6FHkQ,GBTPC53ZrG1ZBY3DT8Mbcw,5.0,Luke,This was our second choice after finding out R...,4.0,4554,4.05814,2009-03-01,second choice finding rio mar closed mardi gra...,...,0.000000,0.000000,242.911519,1156.340135,97.635316,0.000000,69.352156,0.000000,161.813828,339.475310
4,gnEmBGyBs7DOtIXGmJQdKQ,GBTPC53ZrG1ZBY3DT8Mbcw,4.0,Luke,Great oyster happy hours that go on from 5-7pm...,4.0,4554,4.05814,2015-08-19,great oyster happy hours go pm boyfriend share...,...,64.821352,0.000000,77.808380,0.000000,97.635316,342.401188,0.000000,139.886019,98.157071,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52963,CqJxh0TxfV2kb7ny5jeR0g,oBNrLz4EDhiscSlbOl8uAw,4.0,Ruby Slipper - New Orleans,Heard a lot about this place. I was told by ma...,4.5,5193,4.05814,2017-02-20,heard lot place told many including yelp nola ...,...,175.882919,0.000000,84.482656,1143.630113,89.446985,0.000000,533.566941,0.000000,160.091508,0.000000
52964,Qq3hOwfgkJG8CL6B39lHRA,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,I don't even know where to start. This was ou...,4.5,5193,4.05814,2021-12-13,even know start top recommendation local frie...,...,0.000000,0.000000,165.103139,1308.301265,287.237297,0.000000,140.171024,0.000000,237.828579,89.368565
52965,EqXPcN4t6tqKE47tBdUX_Q,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,Long wait and they got my phone number wrong s...,4.5,5193,4.05814,2021-12-04,long wait got phone number wrong even longer e...,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,160.091508,0.000000
52966,HehXCiDy2-vwX94in-_Zcw,oBNrLz4EDhiscSlbOl8uAw,5.0,Ruby Slipper - New Orleans,This place was amazing! From the host taking ...,4.5,5193,4.05814,2022-01-16,place amazing host taking names servers forge...,...,0.000000,0.000000,165.103139,612.217168,183.384019,0.000000,140.171024,0.000000,139.671508,0.000000


In [157]:
# Define features and target variable
X = df.drop(['review_id', 'business_id', 'name', 'text', 'restaurant_avg_star', 'review_count', 'avg_stars10m_radius', 'date', 'clean_text', 'stars'], axis=1)  # Features

y = df['stars']  # Target variable

X.columns

Index(['Customer Service Experience', 'Bar Atmosphere', 'Mediocre Food',
       'Lacking Value', 'New Orleans Culture', 'Long Waits', 'Food Quality',
       'Southern Dishes', 'Shrimp Po' Boy', 'Returning Doubtful',
       'Taste Preferences', 'Southern Comfort Food', 'Happy Hour Selection',
       'Cheese and Steak', 'Philadelphia Cheesesteak', 'Reliable Staff',
       'Quality Dining', 'Outstanding Service', 'Satisfactory Experience',
       'Dissatisfied Feedback', 'Return Intent', 'Southern Classics',
       'Outdoor Seating', 'Service Issues', 'Extraordinary Recommendation',
       'Oyster Specialties', 'Breakfast Choices', 'Favorite Spot', 'Top Picks',
       'Tourist Avoidance'],
      dtype='object')

In [158]:

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform your feature matrix (X)
X_standardized = scaler.fit_transform(X)
X_standardized

array([[ 0.29023303, -0.17749712, -0.14647585, ..., -0.3398965 ,
        -0.19641932, -0.57778414],
       [-0.80616314, -1.24603263, -0.14647585, ..., -0.3398965 ,
         0.05854422, -0.57778414],
       [-0.80616314,  1.10721178, -0.14647585, ..., -0.3398965 ,
         1.98302553,  0.67959163],
       ...,
       [-0.80616314, -1.24603263, -0.14647585, ..., -0.3398965 ,
         0.85277024, -0.57778414],
       [ 0.34834179,  0.73049968, -0.14647585, ..., -0.3398965 ,
         0.61763881, -0.57778414],
       [ 1.10835069,  1.09520258, -0.14647585, ..., -0.3398965 ,
         1.66682837, -0.57778414]])

In [159]:



# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y, test_size=0.2, random_state=42)

# Initialize and train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R-squared
r_squared = r2_score(y_test, y_pred)
print("R-squared:", r_squared)

# Calculate mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Calculate error
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean absolute percentage error:", mape)

R-squared: 0.19116311257705332
Mean Absolute Error: 0.8376612622420733
Mean absolute percentage error: 0.33904041192245893


In [177]:
X_train.sum(axis=0)

array([   4.6082787 ,  101.077742  ,  -17.43733373,   48.1952828 ,
        -78.07043938,  172.60290777,   96.97961934, -109.24441336,
        -59.49741943, -144.2999747 ,   51.29217662,  -39.07212197,
       -134.9889947 ,   11.41540244,    4.3793543 ,  -93.96638823,
        100.84461646,  135.45573365,   92.17503812,  -62.49243945,
        106.17466204,  -36.42636061,   70.08820143,   76.41012994,
         50.51215358,  -36.908919  ,  -55.90924136,   68.68145549,
         32.30286098,  -70.66606078])

In [161]:
coefficients = model.coef_
feature_names = X.columns
df_coefficients = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
# If you want to include the intercept as well
intercept_row = pd.DataFrame({'Feature': ['Intercept'], 'Coefficient': [model.intercept_]})
df_coefficients = pd.concat([intercept_row, df_coefficients], ignore_index=True)
df_coefficients = df_coefficients.sort_values("Coefficient", ascending=False)
df_coefficients.to_csv('Top_Coefficients.csv', index=False)

In [162]:
coefficients_dict = df_coefficients.set_index('Feature')['Coefficient'].to_dict()

In [163]:
coefficients_dict.pop('Intercept')

4.041447546791187

In [164]:
coefficients_dict

{'Extraordinary Recommendation': 0.1987222965720251,
 'Oyster Specialties': 0.12139531294923311,
 'Bar Atmosphere': 0.11608547360545084,
 'Top Picks': 0.09421221225603452,
 'Return Intent': 0.09305972204560804,
 'Breakfast Choices': 0.08996961227116729,
 'Quality Dining': 0.08978076657864845,
 'Lacking Value': 0.08567526466927736,
 'New Orleans Culture': 0.08412170224855198,
 'Cheese and Steak': 0.08392115225686313,
 'Southern Dishes': 0.07290723518667794,
 'Favorite Spot': 0.07130004054646823,
 'Outdoor Seating': 0.0581628476888862,
 'Outstanding Service': 0.056884268681575326,
 'Southern Comfort Food': 0.022213236485093397,
 'Happy Hour Selection': 0.006347162380704644,
 'Southern Classics': 0.005016284199362092,
 'Customer Service Experience': -0.012255811678113397,
 'Reliable Staff': -0.01580811120547814,
 'Dissatisfied Feedback': -0.033248906937029844,
 'Long Waits': -0.047807396288143435,
 'Satisfactory Experience': -0.06341631023634896,
 'Philadelphia Cheesesteak': -0.0710668536

In [178]:
# Get top 5
top_5 = dict(sorted(coefficients_dict.items(), key=lambda item: item[1], reverse=True)[:5])

# Get bottom 5
bottom_5 = dict(sorted(coefficients_dict.items(), key=lambda item: item[1])[:5])


top_5

{'Extraordinary Recommendation': 0.1987222965720251,
 'Oyster Specialties': 0.12139531294923311,
 'Bar Atmosphere': 0.11608547360545084,
 'Top Picks': 0.09421221225603452,
 'Return Intent': 0.09305972204560804}

In [179]:
bottom_5

{'Taste Preferences': -0.2083977418273525,
 'Service Issues': -0.2022651207085938,
 'Tourist Avoidance': -0.1455879199959048,
 'Mediocre Food': -0.14320484400722663,
 'Food Quality': -0.1277631297636556}